In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import matplotlib.pyplot as plt
from matplotlib.pyplot import xticks
from scipy.stats import skew
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
data = pd.read_csv('../input/shot_logs.csv')

In [ ]:
data.head()

In [ ]:
# Best shooters

In [ ]:
# Nba shot accuracy
made = data[data.SHOT_RESULT == 'made']['SHOT_RESULT'].count()
accuracy = round(made / len(data), 2)
accuracy

In [ ]:
# Shot taken per game
shot_taken = data.groupby(['player_name', 'GAME_ID'])['SHOT_RESULT'].count()
shot_taken.head()

In [ ]:
shot_taken = shot_taken.reset_index()
shot_taken = round(shot_taken.groupby('player_name')['SHOT_RESULT'].mean(),2)
shot_taken = shot_taken.reset_index()
shot_taken.columns = ['player_name', 'mean_shot']
shot_taken.head()

In [ ]:
shot_taken.mean_shot.mean()

In [ ]:
# Player has to take at least 9 shots per game 

In [ ]:
players_made = data[data.SHOT_RESULT == 'made'].groupby('player_name')['SHOT_RESULT'].count()
total_shots_player = data.groupby('player_name')['SHOT_RESULT'].count()
accuracy_players = round(players_made / total_shots_player,4) *100

In [ ]:
accuracy_players = accuracy_players.reset_index()

In [ ]:
accuracy_players.columns = ['player_name', 'accuracy']

In [ ]:
accuracy_players['mean_shot_taken']  = shot_taken['mean_shot']

In [ ]:
accuracy_filtered_players = accuracy_players[accuracy_players.mean_shot_taken >= 9]

In [ ]:
accuracy_filtered_players = accuracy_filtered_players.sort_values(['accuracy'], ascending=False)

In [ ]:
accuracy_filtered_players.head()

In [ ]:
# These players are all centers, we just have to check their mean distance shot

In [ ]:
mean_dist_shot = data.groupby('player_name')['SHOT_DIST'].mean()
mean_dist_shot = mean_dist_shot.reset_index()
mean_dist_shot.columns = ['player_name', 'mean_dist_shot']

In [ ]:
accuracy_players['mean_dist_shot'] = mean_dist_shot['mean_dist_shot']

In [ ]:
top_accuracy = accuracy_players[accuracy_players.mean_shot_taken >= 9]
top_accuracy = top_accuracy.sort_values(['accuracy'], ascending=False)
top_accuracy = top_accuracy[:10]
top_accuracy = top_accuracy.sort_values(['player_name'])

In [ ]:
top_accuracy = top_accuracy.reset_index(drop=True)

In [ ]:
top_accuracy

In [ ]:
sns.set(rc={'figure.figsize':(8,6)})

fig = plt.figure()
ax1 = fig.add_subplot(111)
g = sns.barplot(x=top_accuracy.player_name, y=top_accuracy.accuracy ,ax=ax1)
g.set_xticklabels(labels = top_accuracy.player_name,  rotation=90)
ax2 = ax1.twinx()
h = sns.lineplot(x="player_name", y="mean_shot_taken", data=top_accuracy, ax=ax2, color="red")
h.set_xticklabels(labels = top_accuracy.player_name,  rotation=90)
ax2.grid(False)
plt.show()

They all shot at less than 12 feet.
In term of accuracy, the best shooter is Dwight Howard.
The best shooters, taking account of the accuracy and the number of shots taken, are Anthony Davis and Nikola Vucevic

In [ ]:
# Best 3 point shooter

In [ ]:
shot_3 = data[data.SHOT_DIST >=24]

In [ ]:
# 3 point Shot taken per game
shot3_taken = shot_3.groupby(['player_name', 'GAME_ID'])['SHOT_RESULT'].count()
shot3_taken = shot3_taken.reset_index()
shot3_taken = round(shot3_taken.groupby('player_name')['SHOT_RESULT'].mean(),2)
shot3_taken.head()

In [ ]:
shot3_taken.mean()

In [ ]:
shot3_taken = shot3_taken.reset_index()
shot3_taken.columns = ['player_name', 'mean_shot']
shot3_taken.head()

In [ ]:
# Player has to take at least 9 shots per game 
players3_made = shot_3[shot_3.SHOT_RESULT == 'made'].groupby('player_name')['SHOT_RESULT'].count()
total_shots3_player = shot_3.groupby('player_name')['SHOT_RESULT'].count()
accuracy3_players = round(players3_made / total_shots3_player,4) *100

In [ ]:
accuracy3_players = accuracy3_players.reset_index()
accuracy3_players.columns = ['player_name', 'accuracy']
accuracy3_players['mean_shot_taken']  = shot3_taken['mean_shot']

In [ ]:
top_accuracy3 = accuracy3_players[accuracy3_players.mean_shot_taken > 3]
top_accuracy3 = top_accuracy3.sort_values(['accuracy'], ascending=False)

In [ ]:
top_accuracy3 = top_accuracy3[:15]
top_accuracy3 = top_accuracy3.sort_values(['player_name'])
top_accuracy3 = top_accuracy3.reset_index(drop=True)

In [ ]:
top_accuracy3

In [ ]:
sns.set(rc={'figure.figsize':(8,6)})

fig = plt.figure()
ax1 = fig.add_subplot(111)
g = sns.barplot(x=top_accuracy3.player_name, y=top_accuracy3.accuracy ,ax=ax1)
g.set_xticklabels(labels = top_accuracy3.player_name,  rotation=90)
ax2 = ax1.twinx()
h = sns.lineplot(x="player_name", y="mean_shot_taken", data=top_accuracy3, ax=ax2, color="red")
h.set_xticklabels(labels = top_accuracy3.player_name,  rotation=90)
ax2.grid(False)
plt.show()

In term of accuracy, Kyle Korver is the best shooter. 
But, taking account of the accuracy and the number of shots taken, are Stephen Curry and Klay Thompson (the more ou tke shots, the more your accuracy should decrease)

In [ ]:
# The more shots you take, the more your accuracy should decrease
shot_accuracy3 = accuracy3_players[['accuracy', 'mean_shot_taken']]

In [ ]:
shot_accuracy3 = shot_accuracy3.dropna()

In [ ]:
shot_accuracy3 = shot_accuracy3.groupby('mean_shot_taken').mean()

In [ ]:
shot_accuracy3 = shot_accuracy3.reset_index()

In [ ]:
sns.set(rc={'figure.figsize':(10,8)})
sns.scatterplot(x='mean_shot_taken', y= 'accuracy', data=shot_accuracy3)

This means Stephen Curry is the best 3 points shooter with the amount of shots he takes

In [ ]:
# Best defenders

In [ ]:
data.head()

In [ ]:
# Shot taken agianst player per game
shot_defend = data.groupby(['CLOSEST_DEFENDER', 'GAME_ID'])['SHOT_RESULT'].count()
shot_defend.head()

In [ ]:
shot_defend = shot_defend.reset_index()
shot_defend = round(shot_defend.groupby('CLOSEST_DEFENDER')['SHOT_RESULT'].mean(),2)
shot_defend = shot_defend.reset_index()
shot_defend.columns = ['CLOSEST_DEFENDER', 'mean_shot']
shot_defend.head()

In [ ]:
shot_defend.mean_shot.mean()

In [ ]:
shot_defend.mean_shot.max()

In [ ]:
# Player has to defend more than 10 shots

In [ ]:
defender_miss = data[data.SHOT_RESULT == 'missed'].groupby('CLOSEST_DEFENDER')['SHOT_RESULT'].count()
total_shots_defender = data.groupby('CLOSEST_DEFENDER')['SHOT_RESULT'].count()
accuracy_defenders = round(defender_miss / total_shots_defender,4) *100

In [ ]:
accuracy_defenders = accuracy_defenders.reset_index()
accuracy_defenders.columns = ['CLOSEST_DEFENDER', 'accuracy']
accuracy_defenders['mean_shot_defend']  = shot_defend['mean_shot']

In [ ]:
top_def = accuracy_defenders[accuracy_defenders.mean_shot_defend > 10]
top_def = top_def.sort_values(['accuracy'], ascending=False)

In [ ]:
top_def = top_def[:15]
top_def = top_def.sort_values(['CLOSEST_DEFENDER'])

In [ ]:
top_def

In [ ]:
sns.set(rc={'figure.figsize':(8,6)})

fig = plt.figure()
ax1 = fig.add_subplot(111)
g = sns.barplot(x=top_def.CLOSEST_DEFENDER, y=top_def.accuracy ,ax=ax1)
g.set_xticklabels(labels = top_def.CLOSEST_DEFENDER,  rotation=90)
ax2 = ax1.twinx()
h = sns.lineplot(x="CLOSEST_DEFENDER", y="mean_shot_defend", data=top_def, ax=ax2, color="red")
h.set_xticklabels(labels = top_def.CLOSEST_DEFENDER,  rotation=90)
ax2.grid(False)
plt.show()

The best defenders are Serge Ibaka, Terrence Jones, Anthony Davis and Draymond Green

In [ ]:
# Clutch players

In [ ]:
data.head()

In [ ]:
clutch = data[['FINAL_MARGIN', 'PERIOD', 'GAME_CLOCK', 'SHOT_CLOCK', 'PTS_TYPE', 'SHOT_RESULT', 'player_name']]

In [ ]:
# 4th quarter
clutch = clutch[clutch.PERIOD == 4]

In [ ]:
# Convert to datetime
clutch["GAME_CLOCK_SECONDS"] = pd.to_datetime(data["GAME_CLOCK"], format="%M:%S")

In [ ]:
# Convert clock to seconds : for seconds played, 12:00 - x, x as game clock and 12:00 as quarter time
clutch["GAME_CLOCK_SECONDS"] = clutch["GAME_CLOCK_SECONDS"].apply(lambda x: 12*60 - (x.minute * 60 + x.second))

In [ ]:
clutch.head()

In [ ]:
# Clutch = Last 2 minutes, point difference = 5 pts or less.
quarter_duration = 60*12
clutch = clutch[(np.abs(clutch.FINAL_MARGIN) < 5) & (clutch.GAME_CLOCK_SECONDS >= (quarter_duration - 120))]

In [ ]:
clutch.head()

In [ ]:
# Shot taken per game
shot_clutch = clutch.groupby(['player_name'])['SHOT_RESULT'].count()
shot_clutch = shot_clutch.reset_index()
shot_clutch.columns = ['player_name', 'shot_taken']
shot_clutch.head()

In [ ]:
shot_clutch.shot_taken.mean()

In [ ]:
clutch_made = clutch[clutch.SHOT_RESULT == 'made'].groupby('player_name')['SHOT_RESULT'].count()
total_shots_clutch = clutch.groupby('player_name')['SHOT_RESULT'].count()
accuracy_clutch = round(clutch_made / total_shots_clutch,4) *100

In [ ]:
accuracy_clutch = accuracy_clutch.reset_index()
accuracy_clutch.columns = ['player_name', 'accuracy']
accuracy_clutch['shot_taken']  = shot_clutch['shot_taken']

In [ ]:
accuracy_clutch_players = accuracy_clutch[accuracy_clutch.shot_taken >= 7]
accuracy_clutch_players = accuracy_clutch_players.sort_values(['accuracy'], ascending=False)
accuracy_clutch_players = accuracy_clutch_players[:10]
accuracy_clutch_players = accuracy_clutch_players.sort_values(['player_name'])
accuracy_clutch_players = accuracy_clutch_players.reset_index(drop=True)

In [ ]:
accuracy_clutch_players.head()

In [ ]:
sns.set(rc={'figure.figsize':(8,6)})

fig = plt.figure()
ax1 = fig.add_subplot(111)
g = sns.barplot(x=accuracy_clutch_players.player_name, y=accuracy_clutch_players.accuracy ,ax=ax1)
g.set_xticklabels(labels = accuracy_clutch_players.player_name,  rotation=90)
ax2 = ax1.twinx()
h = sns.lineplot(x="player_name", y="shot_taken", data=accuracy_clutch_players, ax=ax2, color="red")
h.set_xticklabels(labels = accuracy_clutch_players.player_name,  rotation=90)
ax2.grid(False)
plt.show()

The best clutch player is Anthony Davis
Taking account of the accuracy and the number of shots taken, James Harden is the best in clutch time (because of the amount of shots he took)

In [ ]:
clutch_accuracy = accuracy_clutch.groupby('shot_taken').mean()
clutch_accuracy = clutch_accuracy.reset_index()

In [ ]:
clutch_accuracy

In [ ]:
sns.set(rc={'figure.figsize':(10,8)})
sns.lineplot(x=clutch_accuracy.shot_taken, y= clutch_accuracy.accuracy)

The pike at 23 shots taken corresponds to James Harden